### This script is adapted from https://github.com/lindawangg/COVID-Net

In [1]:
import numpy as np
import pandas as pd
import os
import random 
from shutil import copyfile
import pydicom as dicom
import cv2

In [2]:
# set parameters here
savepath = 'data'
seed = 0
np.random.seed(seed) # Reset the seed so all runs are the same.
random.seed(seed)
MAXVAL = 255  # Range [0 255]

# path to covid-19 dataset from https://github.com/ieee8023/covid-chestxray-dataset
cohen_imgpath = 'covid-chestxray-dataset/images' 
cohen_csvpath = 'covid-chestxray-dataset/metadata.csv'

# path to covid-14 dataset from https://github.com/agchung/Figure1-COVID-chestxray-dataset
fig1_imgpath = 'Figure1-COVID-chestxray-dataset/images'
fig1_csvpath = 'Figure1-COVID-chestxray-dataset/metadata.csv'

# path to https://www.kaggle.com/c/rsna-pneumonia-detection-challenge
rsna_datapath = 'rsna-pneumonia-detection-challenge'
# get all the normal from here
rsna_csvname = 'stage_2_detailed_class_info.csv' 
# get all the 1s from here since 1 indicate pneumonia
# found that images that aren't pneunmonia and also not normal are classified as 0s
rsna_csvname2 = 'stage_2_train_labels.csv' 
rsna_imgpath = 'stage_2_train_images'

# parameters for COVIDx dataset
train = []
test = []
valid = []
test_count = {'normal': 0, 'pneumonia': 0, 'COVID-19': 0}
train_count = {'normal': 0, 'pneumonia': 0, 'COVID-19': 0}
valid_count = {'normal': 0, 'pneumonia': 0, 'COVID-19': 0}

mapping = dict()
mapping['COVID-19'] = 'COVID-19'
mapping['SARS'] = 'pneumonia'
mapping['MERS'] = 'pneumonia'
mapping['Streptococcus'] = 'pneumonia'
mapping['Klebsiella'] = 'pneumonia'
mapping['Chlamydophila'] = 'pneumonia'
mapping['Legionella'] = 'pneumonia'
mapping['Normal'] = 'normal'
mapping['Lung Opacity'] = 'pneumonia'
mapping['1'] = 'pneumonia'

# train/test split
split = 0.1

# to avoid duplicates
patient_imgpath = {}

In [3]:
# adapted from https://github.com/mlmed/torchxrayvision/blob/master/torchxrayvision/datasets.py#L814
cohen_csv = pd.read_csv(cohen_csvpath, nrows=None)
#idx_pa = csv["view"] == "PA"  # Keep only the PA view
views = ["PA", "AP", "AP Supine", "AP semi erect", "AP erect"]
cohen_idx_keep = cohen_csv.view.isin(views)
cohen_csv = cohen_csv[cohen_idx_keep]

fig1_csv = pd.read_csv(fig1_csvpath, encoding='ISO-8859-1', nrows=None)
#fig1_idx_keep = fig1_csv.view.isin(views)
#fig1_csv = fig1_csv[fig1_idx_keep]

In [4]:
# get non-COVID19 viral, bacteria, and COVID-19 infections from covid-chestxray-dataset
# stored as patient id, image filename and label
filename_label = {'normal': [], 'pneumonia': [], 'COVID-19': []}
count = {'normal': 0, 'pneumonia': 0, 'COVID-19': 0}
for index, row in cohen_csv.iterrows():
    f = row['finding'].split(',')[0] # take the first finding, for the case of COVID-19, ARDS
    if f in mapping: # 
        count[mapping[f]] += 1
        entry = [str(row['patientid']), row['filename'], mapping[f], row['view']]
        filename_label[mapping[f]].append(entry)
        
for index, row in fig1_csv.iterrows():
    if not str(row['finding']) == 'nan':
        f = row['finding'].split(',')[0] # take the first finding
        if f in mapping: # 
            count[mapping[f]] += 1
            if os.path.exists(os.path.join(fig1_imgpath, row['patientid'] + '.jpg')):
                entry = [row['patientid'], row['patientid'] + '.jpg', mapping[f]]
            elif os.path.exists(os.path.join(fig1_imgpath, row['patientid'] + '.png')):
                entry = [row['patientid'], row['patientid'] + '.png', mapping[f]]
            filename_label[mapping[f]].append(entry)
            
print('Data distribution from covid-chestxray-dataset:')
print(count)

Data distribution from covid-chestxray-dataset:
{'normal': 0, 'pneumonia': 33, 'COVID-19': 254}


In [5]:
# add covid-chestxray-dataset into COVIDx dataset
# since covid-chestxray-dataset doesn't have test dataset
# split into train/valid/test by patientid

for key in filename_label.keys():
    arr = np.array(filename_label[key])
    if arr.size == 0:
        continue
    # split by patients
    # num_diff_patients = len(np.unique(arr[:,0]))
    # num_test = max(1, round(split*num_diff_patients))
    # select num_test number of random patients
    if key == 'pneumonia':
        test_patients = ['8','87']
        valid_patients = ['31', '91']
    elif key == 'COVID-19':
        test_patients = ['19', '20', '36', '42', '86', 
                         '94', '97', '117', '132', 
                         '138', '144', '150', '163', '169'] # random.sample(list(arr[:,0]), num_test)
        valid_patients = ['2', '13', '23', '58', '73', 
                         '113', '135', '143', '155', 
                         '161', '168', '179', '186', '194'] # random.sample(list(arr[:,0])
    else: 
        test_patients = []
        valid_patients = []
    print('Key: ', key)
    print('Test patients: ', test_patients)
    print('Valid patients: ', valid_patients)
    # go through all the patients
    for patient in arr:
        if patient[0] not in patient_imgpath:
            patient_imgpath[patient[0]] = [patient[1]]
        else:
            if patient[1] not in patient_imgpath[patient[0]]:
                patient_imgpath[patient[0]].append(patient[1])
            else:
                continue  # skip since image has already been written
        if patient[0] in test_patients:
            if key == 'COVID-19':
                copyfile(os.path.join(cohen_imgpath, patient[1]), os.path.join(savepath, 'test/COVID', patient[1]))
            else:
                copyfile(os.path.join(cohen_imgpath, patient[1]), os.path.join(savepath, 'test/PNEUMONIA', patient[1]))
            test.append(patient)
            test_count[patient[2]] += 1
        elif patient[0] in valid_patients:
            if key == 'COVID-19':
                copyfile(os.path.join(cohen_imgpath, patient[1]), os.path.join(savepath, 'valid/COVID', patient[1]))
            else:
                copyfile(os.path.join(cohen_imgpath, patient[1]), os.path.join(savepath, 'valid/PNEUMONIA', patient[1]))
            valid.append(patient)
            valid_count[patient[2]] += 1
        else:
            if key == 'COVID-19':
                if 'COVID' in patient[0]:
                    copyfile(os.path.join(fig1_imgpath, patient[1]), os.path.join(savepath, 'train/COVID', patient[1]))
                else:
                    copyfile(os.path.join(cohen_imgpath, patient[1]), os.path.join(savepath, 'train/COVID', patient[1]))
            else: 
                copyfile(os.path.join(cohen_imgpath, patient[1]), os.path.join(savepath, 'train/PNEUMONIA', patient[1]))
            train.append(patient)
            train_count[patient[2]] += 1

print('test count: ', test_count)
print('valid count: ', valid_count)
print('train count: ', train_count)

Key:  pneumonia
Test patients:  ['8', '87']
Valid patients:  ['31', '91']
Key:  COVID-19
Test patients:  ['19', '20', '36', '42', '86', '94', '97', '117', '132', '138', '144', '150', '163', '169']
Valid patients:  ['2', '13', '23', '58', '73', '113', '135', '143', '155', '161', '168', '179', '186', '194']
test count:  {'normal': 0, 'pneumonia': 4, 'COVID-19': 31}
valid count:  {'normal': 0, 'pneumonia': 4, 'COVID-19': 30}
train count:  {'normal': 0, 'pneumonia': 25, 'COVID-19': 193}


In [6]:
# add normal and rest of pneumonia cases from https://www.kaggle.com/c/rsna-pneumonia-detection-challenge
csv_normal = pd.read_csv(os.path.join(rsna_datapath, rsna_csvname), nrows=None)
csv_pneu = pd.read_csv(os.path.join(rsna_datapath, rsna_csvname2), nrows=None)
patients = {'normal': [], 'pneumonia': []}

for index, row in csv_normal.iterrows():
    if row['class'] == 'Normal':
        patients['normal'].append(row['patientId'])

for index, row in csv_pneu.iterrows():
    if int(row['Target']) == 1:
        patients['pneumonia'].append(row['patientId'])

for key in patients.keys():
    arr = np.array(patients[key])
    if arr.size == 0:
        continue
    # split by patients 
    #num_diff_patients = len(np.unique(arr))
    #num_test = max(1, round(split*num_diff_patients))
    num_patients = np.load('rsna_test_patients_{}.npy'.format(key)) # random.sample(list(arr), num_test), download the .npy files from the repo.
    # np.save('rsna_test_patients_{}.npy'.format(key), np.array(test_patients))
    test_patients = num_patients[:int(np.floor(num_patients.size)/2)]
    #test_patients = np.load('rsna_test_patients_{}.npy'.format(key)) # random.sample(list(arr), num_test), download the .npy files from the repo.
    #np.save('rsna_test_patients_{}.npy'.format(key), np.array(test_patients))
    valid_patients = num_patients[int(np.ceil(num_patients.size)/2):]
    #valid_patients = np.load('rsna_valid_patients_{}.npy'.format(key)) # random.sample(list(arr), num_test), download the .npy files from the repo.
    #np.save('rsna_valid_patients_{}.npy'.format(key), np.array(valid_patients))
    for patient in arr:
        if patient not in patient_imgpath:
            patient_imgpath[patient] = [patient]
        else:
            continue  # skip since image has already been written
                
        ds = dicom.dcmread(os.path.join(rsna_datapath, rsna_imgpath, patient + '.dcm'))
        pixel_array_numpy = ds.pixel_array
        imgname = patient + '.png'
        if patient in test_patients:
            if key == 'normal':
                cv2.imwrite(os.path.join(savepath, 'test/NORMAL', imgname), pixel_array_numpy)
            else:
                cv2.imwrite(os.path.join(savepath, 'test/PNEUMONIA', imgname), pixel_array_numpy)
            test.append([patient, imgname, key])
            test_count[key] += 1
        elif patient in valid_patients:
            if key == 'normal':
                cv2.imwrite(os.path.join(savepath, 'valid/NORMAL', imgname), pixel_array_numpy)
            else:
                cv2.imwrite(os.path.join(savepath, 'valid/PNEUMONIA', imgname), pixel_array_numpy)
            valid.append([patient, imgname, key])
            valid_count[key] += 1
        else:
            if key == 'normal':
                cv2.imwrite(os.path.join(savepath, 'train/NORMAL', imgname), pixel_array_numpy)
            else:
                cv2.imwrite(os.path.join(savepath, 'train/PNEUMONIA', imgname), pixel_array_numpy)
            train.append([patient, imgname, key])
            train_count[key] += 1

print('test count: ', test_count)
print('valid count: ', valid_count)
print('train count: ', train_count)

test count:  {'normal': 442, 'pneumonia': 299, 'COVID-19': 31}
valid count:  {'normal': 443, 'pneumonia': 298, 'COVID-19': 30}
train count:  {'normal': 7966, 'pneumonia': 5448, 'COVID-19': 193}


In [7]:
# final stats
print('Final stats')
print('Train count: ', train_count)
print('Test count: ', test_count)
print('Valid count: ', valid_count)
print('Total length of train: ', len(train))
print('Total length of test: ', len(test))
print('Total length of valid: ', len(valid))

Final stats
Train count:  {'normal': 7966, 'pneumonia': 5448, 'COVID-19': 193}
Test count:  {'normal': 442, 'pneumonia': 299, 'COVID-19': 31}
Valid count:  {'normal': 443, 'pneumonia': 298, 'COVID-19': 30}
Total length of train:  13607
Total length of test:  772
Total length of valid:  771
